In [1]:
import requests, csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import tqdm
import numpy as np
from collections import OrderedDict

In [168]:
def prod_imdb(imdb_id):
    
    
    #SELECT PRODUCTION
    
    r4 = requests.get("https://www.imdb.com/title/"+imdb_id+"/companycredits")
    bs4 = BeautifulSoup(r4.content)
    
    #SELECT PRODUCERS
    try:
        prod = bs4.find('ul', {'class': 'simpleList'})
        prod_t = prod.text
    except AttributeError : 
        productions = ""
        prod_code = ""
        movie_id = ""
        
    try:
        if "(" in prod_t:
            prod_t = re.sub(r'\([^)]*\)', '', prod_t)
        prod_t = prod_t.replace("\n ", "")
        prod_t = prod_t.strip().split("\n\n")
    
        productions = []
        movie_id = []
        for i in prod_t:
            productions.append(i.strip())
            movie_id.append(imdb_id)
    except UnboundLocalError : 
        production = ""    
        movie_id = ""
        
    #SELECT PRODUCER CODE
    
    try:
        code = prod.find_all("a")
    
        prod_code = []
        for i in range(len(code)):
            prod_code.append((str(code[i]).split("pany/")[1]).split('">')[0])
    except (AttributeError, UnboundLocalError) : 
        prod_code = ""
    
    
    return movie_id, prod_code, productions

In [3]:
df = pd.read_excel('id_title.xlsx')

In [4]:
imdb_id = df["id"].values

In [97]:
columns_name = ["imdb_id", "prod_code", "production"]

In [316]:
prod_df = pd.DataFrame(columns = columns_name)

In [317]:
temp_id = []
temp_code = []
temp_prod = []

In [265]:
imdb_id[39024]

'tt0374916'

In [318]:
for movie in tqdm.tqdm(imdb_id[80000:]):
    temp_id += prod_imdb(movie)[0]
    temp_code += prod_imdb(movie)[1]
    temp_prod += prod_imdb(movie)[2]

100%|██████████| 5991/5991 [3:00:17<00:00,  1.81s/it]  


In [319]:
len(temp_id), len(temp_code),len(temp_prod)

(15600, 15600, 15600)

In [255]:
temp_id[-2]

'tt6739796'

In [307]:
temp1 = temp_id[:-2]
temp2 = temp_code

In [308]:
prod_df["imdb_id"] = temp1
prod_df["prod_code"] = temp2
prod_df["production"] = temp_prod

In [320]:
prod_df["imdb_id"] = temp_id
prod_df["prod_code"] = temp_code
prod_df["production"] = temp_prod

In [321]:
prod_df.to_excel("prod_list2.xlsx")

In [233]:
id_df = pd.DataFrame(columns = ["movie_id"])

In [234]:
code_df = pd.DataFrame(columns = ["prod_code"])

In [235]:
product_df = pd.DataFrame(columns = ["production"])

In [236]:
id_df["movie_id"] = temp_id
code_df["prod_code"] = temp_code
product_df["production"] = temp_prod

In [237]:
id_df.to_excel("id_prod.xlsx")
code_df.to_excel("code_prod.xlsx")
product_df.to_excel("name_prod.xlsx")